In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2' 
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,549 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,02

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
# Create a Spark Dataframe from the dataset
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True)

# Showing the first 5 rows of the DataFrame
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("SELECT year(date) as year, round(avg(price), 2) as avg_price \
                    FROM home_sales \
                    WHERE bedrooms = 4 \
                    GROUP BY year \
                    ORDER BY year")

# show the result
result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

from pyspark.sql.functions import round,avg 

# we filter the data to only include 3 bedrooms and 3 bathrooms
filtered_df = spark.sql("SELECT * FROM home_sales  WHERE bedrooms = 3 AND bathrooms = 3 ")

# group the data by the year the home was built and calculate the average price
avg_price_df = filtered_df.groupBy("date_built").agg(round(avg("price"), 2).alias("avg_price"))

# show the results
avg_price_df.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2016|290555.07|
|      2012|293683.19|
|      2017|292676.79|
|      2014|290852.27|
|      2013|295962.27|
|      2011|291117.47|
|      2015| 288770.3|
|      2010|292859.62|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# Run the SQL query to get the average price for homes with a price greater than or equal to $350,000
result = spark.sql("SELECT view, AVG(price) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view")

# Show the result
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|         avg_price|
+----+------------------+
|   7| 403005.7709497207|
|  51| 788128.2083333334|
|  15|404673.29545454547|
|  54| 798684.8235294118|
|  11|399548.11891891895|
|  29| 397771.6512820513|
|  69| 750537.9411764706|
|  42|396964.50241545896|
|  87|         1072285.2|
|  73| 752861.1764705882|
|  64| 767036.6666666666|
|   3| 398867.5964912281|
|  30|          397862.0|
|  34| 401419.7451923077|
|  59|          791453.0|
|   8|398592.70658682636|
|  28|402124.62176165805|
|  22| 402022.6847826087|
|  85|1056336.7435897435|
|  16| 399586.5311004785|
+----+------------------+
only showing top 20 rows

--- 0.8698854446411133 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.

# Cache the temporary table
spark.sql("CACHE TABLE home_sales")

# Query the cached table
spark.sql("SELECT COUNT(*) FROM home_sales").show()

+--------+
|count(1)|
+--------+
|   33287|
+--------+



In [12]:
# 8. Check if the table is cached.

if spark.catalog.isCached("home_sales"):
    print("home_sales is cached")
else:
    print("home_sales is not cached")

home_sales is cached


In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.

# Calculate average price by view rating
start_time = time.time()
avg_prices = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view HAVING avg_price >= 350000 ORDER BY avg_price DESC")
print("--- Cached runtime: %s seconds ---" % (time.time() - start_time))

# Uncached runtime
start_time = time.time()
avg_prices_uncached = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view HAVING avg_price >= 350000 ORDER BY avg_price DESC")
print("--- Uncached runtime: %s seconds ---" % (time.time() - start_time))

# Show results
avg_prices.show()

--- Cached runtime: 0.054201602935791016 seconds ---
--- Uncached runtime: 0.028162240982055664 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows



In [14]:
# 10. Partition the home sales dataset by the date_built field.
df.write.partitionBy('date_built').parquet('p_home_sales', mode='overwrite')

In [19]:

from pyspark.sql.functions import year

# Read in the CSV data and create a DataFrame
df = spark.read.parquet("p_home_sales")

# Convert the date column to a date type
df = df.withColumn("date", df["date"].cast("date"))

# Add a year column to use for partitioning
df = df.withColumn("year", year("date"))

# Write the DataFrame to disk in Parquet format with partitioning
df.write.partitionBy("year", "date_built").parquet("home_sales_partitioned.parquet")

In [20]:
# 11. Read the formatted parquet data.
df_parquet = spark.read.parquet("home_sales_partitioned.parquet")
df_parquet.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+----------+
|0ffe23e3-9d0f-442...|2021-11-18|429183|       3|        3|       2162|   12567|     1|         0|  49|2021|      2017|
|141ef834-60cf-42e...|2021-10-13|399016|       2|        3|       1773|   13699|     1|         0|  23|2021|      2017|
|2747ff2f-f844-4fa...|2021-12-04|148755|       4|        2|       2471|   10734|     1|         0|  18|2021|      2017|
|4166ee15-9139-4da...|2021-08-20|145303|       2|        2|       2235|    8946|     1|         0|  44|2021|      2017|
|8f49711c-8f7a-443...|2021-10-15|284395|       4|        3|       1741|   11379|     1|         0|  28|2021|      2017|
+--------------------+----------+------+

In [21]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales_partitioned")

In [22]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

from pyspark.sql.functions import avg
from pyspark.sql.types import DecimalType

start_time = time.time()

result = spark.sql("SELECT view, ROUND(AVG(price), 2) as avg_price \
                    FROM home_sales_partitioned \
                    WHERE price >= 350000 \
                    GROUP BY view \
                    HAVING AVG(price) >= 350000 \
                    ORDER BY avg_price DESC")

result.show()

# calculate the runtime
print("Runtime:", time.time() - start_time, "seconds")

end_time = time.time()
print("Runtime: {} seconds".format(end_time - start_time))


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

Runtime: 1.5365118980407715 seconds
Runtime: 1.536665916442871 seconds
--- 1.5368266105651855 seconds ---


In [23]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("home_sales")

In [24]:
# 15. Check if the home_sales is no longer cached

# check if home_sales is cached
if spark.catalog.isCached("home_sales"):
    print("home_sales is cached")
else: 
    print("home_sales is not cached")

home_sales is not cached
